# Multistage Stochastic Unit Commitment Problem

In [40]:
import os
import numpy as np
import pandas as pd
import gurobipy as gp
from scipy import stats, linalg

from sddip import config, utils


## Parameters

In [41]:
test_case_raw_dir = "case6ww/raw"

test_case_raw_dir = os.path.join(config.test_cases_dir, test_case_raw_dir)

bus_file_raw = os.path.join(test_case_raw_dir, "bus_data.txt")
branch_file_raw = os.path.join(test_case_raw_dir, "branch_data.txt")
gen_file_raw = os.path.join(test_case_raw_dir, "gen_data.txt")
gen_cost_file_raw = os.path.join(test_case_raw_dir, "gen_cost_data.txt")
scenario_data_file = os.path.join(test_case_raw_dir, "scenario_data.txt")

bus_df = pd.read_csv(bus_file_raw, delimiter="\s+")
branch_df = pd.read_csv(branch_file_raw, delimiter="\s+")
gen_df = pd.read_csv(gen_file_raw, delimiter="\s+")
gen_cost_df = pd.read_csv(gen_cost_file_raw, delimiter="\s+")

scenario_df = pd.read_csv(scenario_data_file, delimiter="\s+")

In [42]:
nodes = bus_df.bus_i.values.tolist()
edges = branch_df[["fbus", "tbus"]].values.tolist()

graph = utils.Graph(nodes, edges)

ref_bus = bus_df.loc[bus_df.type == 3].bus_i.values[0]

a_inc = graph.incidence_matrix()
b_l = (-branch_df.x /(branch_df.r**2 + branch_df.x**2)).tolist()
b_diag = np.diag(b_l)

m1 = b_diag.dot(a_inc)
m2 = a_inc.T.dot(b_diag).dot(a_inc)

m1 = np.delete(m1, ref_bus-1, 1)
m2 = np.delete(m2, ref_bus-1, 0)
m2 = np.delete(m2, ref_bus-1, 1)

ptdf = m1.dot(np.linalg.inv(m2))

ptdf = np.insert(ptdf, ref_bus-1, 0, axis=1)

ptdf

array([[ 0.        , -0.44078818, -0.37779055, -0.27996603, -0.29542103,
        -0.37975336],
       [ 0.        , -0.3293718 , -0.30667638, -0.53685054, -0.27700207,
        -0.3073835 ],
       [ 0.        , -0.22984002, -0.31553306, -0.18318343, -0.42757689,
        -0.31286315],
       [ 0.        ,  0.06057464, -0.35449287,  0.01854914, -0.10590782,
        -0.2024923 ],
       [ 0.        ,  0.3216443 ,  0.23423126, -0.35271386,  0.11993854,
         0.23695477],
       [ 0.        ,  0.10902536, -0.02083096,  0.0333857 , -0.19061835,
        -0.01678506],
       [ 0.        ,  0.06796751, -0.23669799,  0.02081298, -0.11883341,
        -0.39743076],
       [ 0.        ,  0.06529291,  0.270224  ,  0.01999397, -0.11415718,
         0.14919238],
       [ 0.        , -0.00471827,  0.37528313, -0.00144483,  0.00824936,
        -0.35168468],
       [ 0.        , -0.0077275 , -0.07244512,  0.1104356 , -0.15706353,
        -0.07042873],
       [ 0.        , -0.06324924, -0.13858514, -0.

In [43]:
########################################################################################################################
# Deterministic parameters
########################################################################################################################
gc = np.array(gen_cost_df.c1)
suc = np.array(gen_cost_df.startup)
sdc = np.array( gen_cost_df.startup)
pg_min = np.array(gen_df.Pmin)
pg_max = np.array(gen_df.Pmax)
pl_max = np.array(branch_df.rateA)

n_gens = len(gc)
n_lines, n_buses = ptdf.shape

# Lists of generators at each bus
#
# Example: [[0,1], [], [2]]
# Generator 1 & 2 are located at bus 1
# No Generator is located at bus 2
# Generator 3 is located at bus 3
gens_at_bus = [[] for _ in range(n_buses)]
g = 0
for b in gen_df.bus.values:
    gens_at_bus[b-1].append(g)
    g+=1

# TODO Add ramp rate limits
rg_up_max = np.full(n_gens, 1000)
rg_down_max = np.full(n_gens, 1000)

# TODO Adjust penalty
penalty = 10000
    
########################################################################################################################
# Stochastic parameters
########################################################################################################################
n_nodes_per_stage = scenario_df.groupby("t")["n"].nunique().tolist()
n_stages = len(n_nodes_per_stage)

prob = []
p_d = []

for t in range(n_stages):
    stage_df = scenario_df[scenario_df["t"] == t+1]
    p_d.append(stage_df[scenario_df.columns[scenario_df.columns.to_series().str.contains('Pd')]].values.tolist())
    prob.append(stage_df["p"].values.tolist())


########################################################################################################################
# Expected values of stochastic parameters
########################################################################################################################
ex_pd = [np.array(prob[t]).dot(np.array(p_d[t])) for t in range(n_stages)]

In [44]:
gens_at_bus

[[0], [1], [2], [], [], []]

In [45]:
# prob[t][n]
# Probability of realization n at stage t
#prob

In [46]:
# p_d[t][n][b]
# Demand in stage t and realization n at bus b
#p_d

In [47]:
# ex_pd[t][b]
# Expected demand in stage t at bus b
#ex_pd

## Optimization

### Variables

In [48]:
model = gp.Model("MSUC")

x = {}
y = {}
s_up = {}
s_down = {}
ys_p = {}
ys_n = {}

# TODO Set startup and shutdown costs
# suc = [10]*n_gens
# sdc = [10]*n_gens

for t in range(n_stages):
    for n in range(n_nodes_per_stage[t]):
        for g in range(n_gens):
            x[t,n,g] = model.addVar(vtype = gp.GRB.BINARY, name = f"x_{t+1}_{n+1}_{g+1}")
            y[t,n,g] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"y_{t+1}_{n+1}_{g+1}")
            s_up[t,n,g] = model.addVar(vtype = gp.GRB.BINARY, name = f"s_up_{t+1}_{n+1}_{g+1}")
            s_down[t,n,g] = model.addVar(vtype = gp.GRB.BINARY, name = f"s_down_{t+1}_{n+1}_{g+1}")
        ys_p[t,n] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"ys_p_{t+1}_{n+1}")
        ys_n[t,n] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"ys_n_{t+1}_{n+1}")

model.update()

### Constraints

In [49]:
# Objective
obj = gp.quicksum(prob[t][n]*(gc[g]*y[t,n,g] + suc[g]*s_up[t,n,g] + sdc[g]*s_down[t,n,g] + penalty*(ys_p[t,n] + ys_n[t,n])) 
                    for t in range(n_stages)
                    for n in range(n_nodes_per_stage[t])
                    for g in range(n_gens))

model.setObjective(obj)


# Balance constraints
model.addConstrs((gp.quicksum(y[t,n,g] for g in range(n_gens)) + ys_p[t,n] - ys_n[t,n] == gp.quicksum(p_d[t][n])
                    for t in range(n_stages)
                    for n in range(n_nodes_per_stage[t])),                    
                    "balance")


# Generator constraints
model.addConstrs((y[t,n,g] >= pg_min[g]*x[t,n,g] 
                    for g in range(n_gens)
                    for t in range(n_stages) 
                    for n in range(n_nodes_per_stage[t])),
                    "min-generation")

model.addConstrs((y[t,n,g] <= pg_max[g]*x[t,n,g]  
                    for g in range(n_gens)
                    for t in range(n_stages) 
                    for n in range(n_nodes_per_stage[t])),
                    "max-generation")


# Power flow constraints
for t in range(n_stages):
    for n in range(n_nodes_per_stage[t]):
        line_flows = [gp.quicksum(ptdf[l,b] * (gp.quicksum(y[t,n,g] for g in gens_at_bus[b]) - p_d[t][n][b])
                        for b in range(n_buses)) for l in range(n_lines)]
        model.addConstrs((line_flows[l] <= pl_max[l] for l in range(n_lines)), "power-flow(1)")
        model.addConstrs((-line_flows[l] <= pl_max[l] for l in range(n_lines)), "power-flow(2)")


# Startup shutdown constraints
# t=0
x_init = [0]*n_gens
model.addConstrs((x[0,0,g] - x_init[g] <= s_up[0,0,g] for g in range(n_gens)), "up-down(1)")
model.addConstrs((x[0,0,g] - x_init[g] == s_up[0,0,g] - s_down[0,0,g] for g in range(n_gens)), "up-down(2)")
# t>0
for t in range(1,n_stages):
    for n in range(n_nodes_per_stage[t]):
        model.addConstrs((x[t,n,g] - x[t-1,n_prev,g] <= s_up[t,n,g] 
                        for g in range(n_gens) 
                        for n_prev in range(n_nodes_per_stage[t-1])), 
                        "up-down(1)")
        model.addConstrs((x[t,n,g] - x[t-1,n_prev,g] <= s_up[t,n,g] - s_down[t,n,g] 
                        for g in range(n_gens)
                        for n_prev in range(n_nodes_per_stage[t-1])), 
                        "up-down(2)")


# Ramp rate constraints
# t=0
y_init = [0]*n_gens
model.addConstrs((y[0,0,g] - y_init[g] <= rg_up_max[g] for g in range(n_gens)), "rate-up")
model.addConstrs((y_init[g] - y[0,0,g] <= rg_down_max[g] for g in range(n_gens)), "rate-down(2)")
# t>0
for t in range(1,n_stages):
    for n in range(n_nodes_per_stage[t]):
        model.addConstrs((y[t,n,g] - y[t-1,n_prev,g] <= rg_up_max[g]
                        for g in range(n_gens)
                        for n_prev in range(n_nodes_per_stage[t-1])), 
                        "rate-up")
        model.addConstrs((y[t-1,n_prev,g] - y[t,n,g] <= rg_down_max[g]
                        for g in range(n_gens)
                        for n_prev in range(n_nodes_per_stage[t-1])), 
                        "rate-down")


model.update()
#model.display()

### Solve

In [50]:
#model.setParam("OutputFlag",0)

model.optimize()

#model.setParam("OutputFlag",1)

model.printAttr("X")
print()
print(f"Optimal value: {obj.getValue()}")

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 944 rows, 224 columns and 2284 nonzeros
Model fingerprint: 0xcd68103e
Variable types: 80 continuous, 144 integer (144 binary)
Coefficient statistics:
  Matrix range     [5e-03, 2e+02]
  Objective range  [3e+00, 3e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-01, 1e+03]
Presolve removed 671 rows and 67 columns
Presolve time: 0.00s
Presolved: 273 rows, 157 columns, 764 nonzeros
Variable types: 64 continuous, 93 integer (93 binary)
Found heuristic solution: objective 12747.833756

Root relaxation: objective 1.208372e+04, 176 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 12083.7179    0   36 12747.8338 12083.7179  5.21%     -    0s
H    0     0                   

In [51]:
x_out = [f"x[{t+1},{n+1},{g+1}]:  {x[t,n,g].x}" for t in range(n_stages) for n in range(n_nodes_per_stage[t]) for g in range(n_gens)]
y_out = [f"y[{t+1},{n+1},{g+1}]:  {y[t,n,g].x}" for t in range(n_stages) for n in range(n_nodes_per_stage[t]) for g in range(n_gens)]
s_up_out = [f"s_up[{t+1},{n+1},{g+1}]:  {s_up[t,n,g].x}" for t in range(n_stages) for n in range(n_nodes_per_stage[t]) for g in range(n_gens)]
s_down_out = [f"s_down[{t+1},{n+1},{g+1}]:  {s_down[t,n,g].x}" for t in range(n_stages) for n in range(n_nodes_per_stage[t]) for g in range(n_gens)]

print("Commitment decisions")
for text in x_out:
    print(f"{text}")

print("Dispatch decisions")
for text in y_out:
    print(f"{text}")

print("Startup decisions")
for text in s_up_out:
    print(f"{text}")

print("Shutdown decisions")
for text in s_down_out:
    print(f"{text}")

Commitment decisions
x[1,1,1]:  -0.0
x[1,1,2]:  1.0
x[1,1,3]:  0.0
x[2,1,1]:  0.0
x[2,1,2]:  1.0
x[2,1,3]:  0.0
x[2,2,1]:  0.0
x[2,2,2]:  1.0
x[2,2,3]:  -0.0
x[2,3,1]:  0.0
x[2,3,2]:  1.0
x[2,3,3]:  0.0
x[3,1,1]:  0.0
x[3,1,2]:  1.0
x[3,1,3]:  1.0
x[3,2,1]:  0.0
x[3,2,2]:  1.0
x[3,2,3]:  1.0
x[3,3,1]:  0.0
x[3,3,2]:  1.0
x[3,3,3]:  1.0
x[4,1,1]:  0.0
x[4,1,2]:  1.0
x[4,1,3]:  1.0
x[4,2,1]:  0.0
x[4,2,2]:  1.0
x[4,2,3]:  1.0
x[4,3,1]:  0.0
x[4,3,2]:  1.0
x[4,3,3]:  1.0
x[5,1,1]:  0.0
x[5,1,2]:  1.0
x[5,1,3]:  1.0
x[5,2,1]:  0.0
x[5,2,2]:  1.0
x[5,2,3]:  1.0
x[5,3,1]:  0.0
x[5,3,2]:  1.0
x[5,3,3]:  1.0
x[6,1,1]:  0.0
x[6,1,2]:  1.0
x[6,1,3]:  1.0
x[6,2,1]:  0.0
x[6,2,2]:  1.0
x[6,2,3]:  1.0
x[6,3,1]:  0.0
x[6,3,2]:  1.0
x[6,3,3]:  1.0
Dispatch decisions
y[1,1,1]:  0.0
y[1,1,2]:  86.48695926086106
y[1,1,3]:  0.0
y[2,1,1]:  0.0
y[2,1,2]:  126.87126404152701
y[2,1,3]:  0.0
y[2,2,1]:  0.0
y[2,2,2]:  117.01154183547018
y[2,2,3]:  0.0
y[2,3,1]:  0.0
y[2,3,2]:  120.16666083335073
y[2,3,3]:  0.0

In [52]:
costs = []
c = 0
for t in reversed(range(n_stages)):
    for n in range(n_nodes_per_stage[t]):
        for g in range(n_gens):
            c += prob[t][n] * y[t,n,g].x*gc[g]
    costs.append(c)


costs

[2241.6477674386692,
 4346.984095900581,
 6493.171785613659,
 8555.256292646112,
 9809.164005818788,
 10702.833755861266]